In [0]:
%pip install databricks-labs-dqx==0.6.0


In [0]:
dbutils.library.restartPython()


In [0]:
import json
from glob import glob

paths = glob('../layer_02_silver/*.json')
tables = [path.split('/')[-1].split('.')[0] for path in paths]
tables.sort()
tables.append('None Selected')

dbutils.widgets.dropdown('table', 'None Selected', tables)


In [0]:
table_name = dbutils.widgets.get('table')


In [0]:
import os
import sys
import json

sys.path.append(f"{os.getcwd()}/..")

from databricks.sdk import WorkspaceClient
from databricks.labs.dqx.engine import DQEngineCore
from databricks.labs.dqx.profiler.generator import DQGenerator

if table_name == 'None Selected':
    print('No table selected.')
else:
    ws = WorkspaceClient()
    dq = DQEngineCore(ws, spark)
    df = spark.table(f'silver.{table_name}')
    _, profiles = dq.profile(df)
    generator = DQGenerator(ws)
    checks = generator.generate_dq_rules(profiles)
    json_str = json.dumps(checks, indent=4)
    displayHTML(f"<pre>{json_str}</pre>")
